
# 📦 Dataset & DataLoader in PyTorch

---

## ❌ Problems with Naive (Full-Batch) Training

Training on the **entire dataset at once** causes:

1. **Memory inefficiency**

   * Entire dataset must fit in RAM / GPU memory.

2. **Poor convergence behavior**

   * Updates happen rarely → slower & unstable learning.

### ✅ Solution

👉 **Train using mini-batches** instead of full data.

---

## 🧠 Simple Manual Mini-Batch Approach (Why It’s Not Ideal)

You *can* manually create batches using loops and slicing:

* Loop over epochs
* Slice tensors into chunks of `batch_size`
* Forward → loss → backward → optimizer step

### ⚠️ Problems with this approach

1. ❌ No standard interface for data
2. ❌ No easy way to apply transformations
3. ❌ No built-in shuffling & sampling
4. ❌ No parallel data loading
5. ❌ Batch logic clutters training loop

👉 **This is why PyTorch gives us Dataset & DataLoader**

---

## 🧩 Core Idea: Dataset vs DataLoader

> **Dataset** = *What is the data & how to get one sample*
> **DataLoader** = *How to iterate efficiently over the data*

---



## 📘 Dataset Class (Blueprint for Data)

The **Dataset** class (`torch.utils.data.Dataset`) defines **how data is accessed** (how to read data and its size.).

### Must implement 3 methods:

```python
__init__()      # how data is loaded Initialize data, file paths, or transforms
__len__()       # total number of samples
__getitem__(i) # returns sample (and label) at index i
```

### Key Intuition

* Dataset knows **nothing about batching**
* It only knows **“given an index, return one sample”**

---



```
# Example Structure
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, index):
        # Apply transforms here if needed
        return self.features[index], self.labels[index]
```

## 🚚 DataLoader Class (Data Pipeline Manager)

The **DataLoader Class (`torch.utils.data.DataLoader`) wraps a Dataset** and handles:

* ✅ Batching
* ✅ Shuffling
* ✅ Parallel loading
* ✅ Efficient iteration

### Typical Training Loop

```python
for batch_x, batch_y in dataloader:
    # forward
    # loss
    # backward
    # optimizer step
```

➡️ Clean, readable, production-ready

---

## 🔁 DataLoader Control Flow (Per Epoch)

1. **Shuffle indices**: At the start of an epoch, indices are shuffled (if `shuffle=True`)
2. **Chunking**: Indices are divided into chunks of `batch_size`.
3. For each batch:

   * **Fetching**: Samples are fetched from the Dataset using `__getitem__` for the indices in the chunk
   * Collect samples
   * **Collation**: Samples are combined into a single batch (using `collate_fn`).
4. **Yield**: The batch is returned to the training loop.

---



## ⚙️ Parallel Data Loading (`num_workers`)

DataLoader can use multiple processes (`num_workers`) to speed up data retrieval.

### Example Setup

* Total samples = **10,000**
* Batch size = **32**
* num_workers = **4**
* ≈ **312 batches / epoch**

### What Happens?

* Main process creates batch index lists
* Workers load batches **in parallel**
* While GPU trains on batch 1:

  * Workers prepare batch 2, 3, 4…
* Result: **Pipeline effect → no idle GPU**

✅ Faster training
✅ Better CPU utilization



---

## 🎯 Samplers (How Indices Are Chosen)

Samplers determine the strategy for drawing samples from the dataset.

### Built-in Samplers

1. **SequentialSampler**

   * Samples in order: `0, 1, 2, ...`
   * Default when `shuffle=False`

2. **RandomSampler**

   * Random order (no replacement)
   * Default when `shuffle=True`
3. **Custom Samplers**
   * Useful for handling imbalanced datasets or specific requirements (e.g., time series).

### Why Custom Samplers?

* Imbalanced datasets
* Time-series data
* Class-balanced sampling

---



## 🧩 `collate_fn` (How Samples Become a Batch)

### What is `collate_fn`?

A function that **combines individual samples into a batch**.

* Default: simple stacking
* Custom: padding, masking, special formats

### Common Use Case: NLP Padding

Different sentence lengths → need padding before batching.

```text
[1, 2, 3]
[4, 5]
[6, 7, 8, 9]
```

➡️ `collate_fn` pads them to equal length

---



## ⚙️ Important DataLoader Parameters

| Parameter     | Meaning                    |
| ------------- | -------------------------- |
| `dataset`     | The source dataset (must implement `__len__`, `__getitem__)` |
| `batch_size`  | Number of samples per batch. Larger sizes speed up GPU training but use more memory.          |
| `shuffle`     | If `True`, reshuffles data every epoch. Crucial to prevent model bias based on order.   |
| `num_workers` | Number of subprocesses for loading. Set `>0` to utilize multi-core CPUs and reduce bottlenecks.|
| `pin_memory`  | If `True`, copies tensors to pinned (page-locked) memory. Improves transfer speed to CUDA (GPU) devices.|
| `drop_last`   | If `True`, drops the last incomplete batch if dataset size isn't divisible by batch size (useful for Batch Norm).|
| `collate_fn`  | Custom batch creation      |
| `sampler`     | Custom sampling logic      |

---



## 🧠 Mental Model (Very Important)

```
Dataset        → defines "ONE sample"
DataLoader     → defines "HOW batches flow"
Sampler        → defines "WHICH samples"
collate_fn     → defines "HOW samples combine"
num_workers    → defines "HOW FAST data arrives"
```

---

## ✅ Why This Matters (Exam + Production)

* Clean separation of concerns
* Scales to large datasets
* Enables GPU-efficient training
* Industry-standard PyTorch pattern

---

In [48]:
from sklearn.datasets import make_classification
import torch
from torch.utils.data import Dataset, DataLoader

In [49]:
# Step 1: Create a synthetic classification dataset using sklearn
# This mimics having raw data (like a CSV) before it enters the PyTorch pipeline.
# X -> feature matrix (inputs)
# y -> target labels (outputs)
X, y = make_classification(
    n_samples=10,       # Total number of rows (samples) in our dummy dataset
    n_features=2,       # Number of columns (input features) per sample
    n_informative=2,    # Features that actually predict the class
    n_redundant=0,      # Features that are random noise
    n_classes=2,        # Binary classification (0 or 1)
    random_state=42     # Ensures we get the exact same random numbers every time
)

In [50]:
print(X) # View feature matrix

[[ 1.06833894 -0.97007347]
 [-1.14021544 -0.83879234]
 [-2.8953973   1.97686236]
 [-0.72063436 -0.96059253]
 [-1.96287438 -0.99225135]
 [-0.9382051  -0.54304815]
 [ 1.72725924 -1.18582677]
 [ 1.77736657  1.51157598]
 [ 1.89969252  0.83444483]
 [-0.58723065 -1.97171753]]


In [51]:
print(y) # View labels

[1 0 0 0 0 1 1 1 1 0]


In [52]:
# Display raw shapes for verification
# X shape will be (10, 2) and y shape will be (10,)
print(f"Shape of X: {X.shape}") # Shape of X -> (number_of_samples, number_of_features)
print(f"Shape of y: {y.shape}") # Shape of y -> (number_of_samples,)

Shape of X: (10, 2)
Shape of y: (10,)


In [53]:
# Step 2: Convert the data to PyTorch tensors
# PyTorch models require inputs as Tensors, not numpy arrays.
# float32 is standard for features; long (int64) is standard for classification labels.
X = torch.tensor(X, dtype=torch.float32) # Convert features to float tensor (required for neural networks)
y = torch.tensor(y, dtype=torch.long) # Convert labels to integer tensor
# dtype=torch.long is required for classification loss functions

In [54]:
# Inspect tensors
X

tensor([[ 1.0683, -0.9701],
        [-1.1402, -0.8388],
        [-2.8954,  1.9769],
        [-0.7206, -0.9606],
        [-1.9629, -0.9923],
        [-0.9382, -0.5430],
        [ 1.7273, -1.1858],
        [ 1.7774,  1.5116],
        [ 1.8997,  0.8344],
        [-0.5872, -1.9717]])

In [55]:
y

tensor([1, 0, 0, 0, 0, 1, 1, 1, 1, 0])

In [56]:
# The Dataset class acts as a blueprint. It decouples *how* we store/read data
# from *how* we use it in training.
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        """
        Initializes the dataset.
        Here we simply store the tensors we created above.
        In a real scenario, you might store file paths here instead of loading everything to RAM.
        """
                # Store features and labels

        self.features = features
        self.labels = labels

    def __len__(self):
        """
        Returns the total number of samples in the dataset.
        DataLoader uses this to calculate how many batches are in an epoch.
        """
                # Return total number of samples

        return self.features.shape[0]   # Number of rows in features tensor

    def __getitem__(self, index):
        """
        Retrieves a single sample (features + label) at the specific 'index'.
        This method is called repeatedly by the DataLoader to construct a batch.
        """
                # Return ONE sample (feature, label) at given index

        return self.features[index], self.labels[index]

In [57]:
# Instantiate the custom dataset
dataset = CustomDataset(X, y) # Create dataset object

In [58]:
# Verification:  Total number of samples in dataset
print(f"Dataset Length: {len(dataset)}") # Should be 10
print(f"Sample at index 2: {dataset[2]}") # Returns tuple (tensor([x1, x2]), tensor(label))

Dataset Length: 10
Sample at index 2: (tensor([-2.8954,  1.9769]), tensor(0))


In [59]:
# Get the 3rd sample (index = 2)
dataset[2]

(tensor([-2.8954,  1.9769]), tensor(0))

In [60]:
# Convert the data to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

/tmp/ipython-input-392191959.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.float32)
/tmp/ipython-input-392191959.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y, dtype=torch.long)


In [61]:
# The DataLoader wraps the Dataset to handle batching automatically.
# batch_size=2: It will fetch 2 samples at a time.
# shuffle=False: It will take samples sequentially (Index 0,1 then 2,3...).
dataloader = DataLoader(
    dataset,
    batch_size=2, # Number of samples per batch
    shuffle=False # Do not shuffle (keeps order)
)

# Iterate through batches
print("\nStarting Iteration over DataLoader:")
print("="*50)


Starting Iteration over DataLoader:


In [62]:
# This loop simulates one 'epoch' of training.
# The DataLoader handles the complex logic of grabbing indices, fetching data via __getitem__,
# and collating them into batches .
for batch_features, batch_labels in dataloader:
    # batch_features shape: [2, 2] (batch_size, n_features)
    # batch_labels shape:   [2]    (batch_size)
    print(f"Batch Features:\n{batch_features}")
    print(f"Batch Labels:\n{batch_labels}")
    print("-" * 50)

# Since we have 10 samples and a batch size of 2, this loop runs exactly 5 times (10/2 = 5).

Batch Features:
tensor([[ 1.0683, -0.9701],
        [-1.1402, -0.8388]])
Batch Labels:
tensor([1, 0])
--------------------------------------------------
Batch Features:
tensor([[-2.8954,  1.9769],
        [-0.7206, -0.9606]])
Batch Labels:
tensor([0, 0])
--------------------------------------------------
Batch Features:
tensor([[-1.9629, -0.9923],
        [-0.9382, -0.5430]])
Batch Labels:
tensor([0, 1])
--------------------------------------------------
Batch Features:
tensor([[ 1.7273, -1.1858],
        [ 1.7774,  1.5116]])
Batch Labels:
tensor([1, 1])
--------------------------------------------------
Batch Features:
tensor([[ 1.8997,  0.8344],
        [-0.5872, -1.9717]])
Batch Labels:
tensor([1, 0])
--------------------------------------------------
